## Importing necessary libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
# !pip install -q pmdarima

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use('fivethirtyeight')
import pathlib
import os
import seaborn as sns
import pandas as pd
from datetime import datetime

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
# import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX
# import joblib
sns.set()

## Mounting Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# root_dir = "/content/gdrive/My Drive/"
# base_dir = root_dir + 'MachineLearning/DelhiTemperaturePrediction/'

In [ ]:
# def displayDirContent(dir):
#   if pathlib.posixpath.exists(dir):
#     for name in list(pathlib.Path(dir).glob('*')):
#       print(name)
#   else:
#     print("Path does not exists")

In [ ]:
# displayDirContent(base_dir)

## Loading Data

In [ ]:
print(os.listdir("../input/delhi-weather-data"))

In [ ]:
data_dir = "../input/delhi-weather-data/"
# displayDirContent(data_dir)

In [ ]:
data = pd.read_csv(data_dir + 'testset.csv')

## Overview to the data

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
def overViewOfTheData(data,frows=5,lrows=5):
  print("Shape: ",data.shape,"\n\n")
  
  print("Columns: ",data.columns,"\n\n")

  print("Info : ")
  print(data.info())

In [ ]:
overViewOfTheData(data)

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(x=data.count()[:],y=data.count().index)
plt.xlabel('Non-Null Values Count')
plt.ylabel('Features')

*   Many of the features have a lot of missing values
*   We will remove those features which are having a lot of missing values (heatindexm, precipm, wgustm & windchillm) & we will try to fill missing values in the rest of the features
*   We can see there are some missing values in **temp** also.


In [ ]:
data = data.drop([' _heatindexm',' _precipm',' _wgustm',' _windchillm'],axis=1)

## Pre-processing and EDA

In [ ]:
# Date-Time column is not in the desired format. So, first we will convert it into the desired format (yyyy-mm-dd HH:MM)
# And the we will make that column the index of the data

data['datetime_utc'] = pd.to_datetime(data['datetime_utc'].apply(lambda x: datetime.strptime(x,"%Y%m%d-%H:%M").strftime("%Y-%m-%d %H:%M")))
data['datetime_utc'].head()

In [ ]:
data = data.set_index('datetime_utc',drop=True)
data.index.name = 'datetime'

In [ ]:
fig, ax = plt.subplots()
data[' _tempm'].plot(figsize=(15,12),ax=ax)
ax.set_xlabel('Date-Time')
ax.set_ylabel('Temperature in C')
ax.set_title('Temperature in Delhi')
plt.show()

*  We can a **seasonal** pattern in the timeseries
*  It is also not continuous as it is having some missing data (ex: between 2000 and 2001)
*  This could be a problem while modeling the timeseries. So, to avoid this we will train our model on the data from the year-2001 as we have enough data for training and there is no significant trend that we will miss by dropping the data before 2001
*  We have some **OUTLIERS** in the series also as we can see some really high temperature values. We will remove these outliers.
*  We have large amount of data but we will only use the necessary data (2013-2016)

In [ ]:
# Dropping the data before 2001
data = data['2001':]

In [ ]:
# We will remove the missing data and later we will interpolate the temperature for that missing data
print("Before : ", data.shape)
data.dropna(subset=[' _tempm'],inplace=True)
print("After :", data.shape)

In [ ]:
data.index.minute.value_counts()

*  Here we can see we have **irregular time-intervals**
*  So we will remove the minute time-stamp and will only consider the hourly data

In [ ]:
categoricalColumns = list(set(data.columns) - set(data._get_numeric_data().columns))
categoricalColumns

In [ ]:
# We are resampling it by hours & filling the missing values using the interpolation method
# Notice here we will only get numeric columns so we will have to add the categorical columns additionaly
newdata = data.resample('H').mean().interpolate()
newdata.info()

In [ ]:
# To resample the categorical data we will consider the firt observation and to fill the missing values we will use ffill method
newdata[list(categoricalColumns)] = data[categoricalColumns].resample('H').first().ffill().head()
newdata.head()

In [ ]:
def plotAggregateValues(data,column=None):
  if column in data.columns:
    plt.figure(figsize = (18,25))
    
    ax1 = plt.subplot(4,2,1)
    newdata[column].groupby(newdata.index.year).mean().plot(ax=ax1,title='yearly mean values')
    ax1.set_xlabel('years')
    ax1.set_ylabel(column)
  
    ax2 = plt.subplot(4,2,2)
    newdata[column].groupby(newdata.index.month).mean().plot(ax=ax2,title='monthly mean values')
    ax2.set_xlabel('months')
    ax2.set_ylabel(column)

    # ax3 = plt.subplot(4,2,3)
    # newdata[column].groupby(newdata.index.weekday).mean().plot(ax=ax3,title='weekdays mean values')
    # ax3.set_xlabel('weekdays')
    # ax3.set_ylabel(column)

    ax4 = plt.subplot(4,2,4)
    newdata[column].groupby(newdata.index.hour).mean().plot(ax=ax4,title='hourly mean values')
    ax4.set_xlabel('hours')
    ax4.set_ylabel(column)

    # ax5 = plt.subplot(4,2,5)
    # newdata[column].groupby(newdata.index.minute).mean().plot(ax=ax5,title='minute wise mean values')
    # ax5.set_xlabel('minutes')
    # ax5.set_ylabel(column)

    # ax6 = plt.subplot(4,2,6)
    # newdata[column].groupby(newdata.index.second).mean().plot(ax=ax6,title='seconds wise mean values')
    # ax6.set_xlabel('seconds')
    # ax6.set_ylabel(column)

  else:
    print("Column name not specified or Column not in the data")

In [ ]:
plotAggregateValues(newdata,' _tempm')

*  We can see highest temperature during 5th & 6th month as it is summer time and low temperature during the end and start of the year because of winter.
*  Also, there is high temperature during 11-13 hours as it is noon time and low temperature during night hours.

In [ ]:
def plotBoxNdendity(data,col=None):
  if col in data.columns:    
    plt.figure(figsize=(18,8))

    ax1 = plt.subplot(121)
    data.boxplot(col,ax=ax1)
    ax1.set_ylabel('Boxplot temperature levels in Delhi', fontsize=10)

    ax2 = plt.subplot(122)
    data[col].plot(ax=ax2,legend=True,kind='density')
    ax2.set_ylabel('Temperature distribution in Delhi', fontsize=10)

  else:
    print("Column not in the data")

In [ ]:
plotBoxNdendity(data,' _tempm')

*  We can observe outliers in box plot which are extremely high.
*  50% of the temperature values are distributed around ~26 C

### Train & Test Split

In [ ]:
train = newdata[:'2015']
test = newdata['2016':]


## Model Building 

### 1. Identification

In [ ]:
# Let's decompose the time series to visualize trend, season and noise seperately
def decomposeNplot(data):
  decomposition = sm.tsa.seasonal_decompose(data)

  plt.figure(figsize=(15,16))

  ax1 = plt.subplot(411)
  decomposition.observed.plot(ax=ax1)
  ax1.set_ylabel('Observed')

  ax2 = plt.subplot(412)
  decomposition.trend.plot(ax=ax2)
  ax2.set_ylabel('Trend')

  ax3 = plt.subplot(413)
  decomposition.seasonal.plot(ax=ax3)
  ax3.set_ylabel('Seasonal')

  ax4 = plt.subplot(414)
  decomposition.resid.plot(ax=ax4)
  ax4.set_ylabel('Residuals')

  return decomposition

In [ ]:
# Resampling the data to mothly and averaging out the temperature & we will predict the monthly average temperature
ftraindata = train[' _tempm'].resample('M').mean()
ftestdata = test[' _tempm'].resample('M').mean()

In [ ]:
# Taking the seasonal difference S=12 and decomposing the timeseries
decomposition = decomposeNplot(ftraindata.diff(12).dropna())

#### Stationary?

In [ ]:
# Let's check for stationarity (Augmented Dickey Fuller test)
results = adfuller(ftraindata.diff(12).dropna())
results

* p-value is less than 0.05 and test-statistic is also less very -ev
* So we can say the series is stationary and we can model it without any further transforms 

#### Seasonal?

*  We observed before that there is a yearly periodic pattern -> Seasonal

#### Order of the model?

In [ ]:
# To get non-seasonal oreders of the SARIMAX Model we will first use ACF & PACF plots
plt.figure(figsize=(10,8))

ax1 = plt.subplot(211)
acf = plot_acf(ftraindata.diff(12).dropna(),lags=30,ax=ax1)

ax2 = plt.subplot(212)
pacf = plot_pacf(ftraindata.diff(12).dropna(),lags=30,ax=ax2)

*  It's hard to get the idea of the non-seasonal orders from these plots 

In [ ]:
# To get seasonal oreders of the SARIMAX Model we will first use ACF & PACF plots at seasonal lags 

lags = [12*i for i in range(1,4)]

plt.figure(figsize=(10,8))

ax1 = plt.subplot(211)
acf = plot_acf(ftraindata.diff(12).dropna(),lags=lags,ax=ax1)

ax2 = plt.subplot(212)
pacf = plot_pacf(ftraindata.diff(12).dropna(),lags=lags,ax=ax2)

*  As ACF cuts off after lag 1 & PACF is trailing off we can say that the order of seasonal MA is 1 (Q=1)

### 2. Estimation 

In [ ]:
model = SARIMAX(ftraindata,order=(0,0,1),seasonal_order=(0,1,1,12),trend='n')
results = model.fit()

#### Automatic Model Selection

In [ ]:
# # Lets select the best model based on the aic & bic scores using auto_arima
# results = pm.auto_arima(ftraindata,
#                       seasonal=True, m=12,
#                       d=0,D=1,trace=True,
#                       error_action='ignore',
#                       suppress_warnings=True)

### 3. Diagnostics

In [ ]:
# Check the value of Prob(Q) if it is > 0.05 => The residuals are uncorrelated
# Similarly if Prob(JB) > 0.05 => The residuals are normally distributed
results.summary()

In [ ]:
# Mean Absolute Error for training data
print(np.mean(np.abs(results.resid)))

* ~3 *C monthly average temperature error

In [ ]:
diagnostics = results.plot_diagnostics(figsize=(10,10))

*  Here we can see:   
  1. Standardized residual plot: No obvious structure ✔
  2. Histogram & KDE: KDE is normally distributed ✔
  3. Normal Q-Q: Almost all the points are on the red line ✔
  4. Correlogram of residuals: is nearly zero for all lags ✔ 

### 4. Forecasting

In [ ]:
forecast = results.get_forecast(steps=len(ftestdata))

In [ ]:
predictedmean = forecast.predicted_mean
bounds = forecast.conf_int()
lower_limit = bounds.iloc[:,0]
upper_limit = bounds.iloc[:,1]

In [ ]:
plt.figure(figsize=(12,8))

plt.plot(ftraindata.index, ftraindata, label='train')
plt.plot(ftestdata.index,ftestdata,label='actual')

plt.plot(predictedmean.index, predictedmean, color='r', label='forecast')

plt.fill_between(lower_limit.index,lower_limit,upper_limit, color='pink')

plt.xlabel('Date')
plt.ylabel('Delhi Temperature')
plt.legend()
plt.show()

### 5. Saving the model

In [ ]:
# displayDirContent(base_dir)

In [ ]:
# filename = 'SARIMA_0_0_1_0_1_1_12.pkl'
# joblib.dump(results,filename = base_dir + 'Models/' + filename)